## The k-NN algorithm

Task: Identify the Nearest Neighbors

### 1. Concept

1.1. <ins>Assumption</ins>: Similar inputs have similar outputs

1.2. <ins>Summary</ins>: For a test input $x$, the most common label among its $k$ most similar training samples, is returned as its output label

1.3. <ins>Formal definition</ins>:

> Test point : $x$<br>
> Training Data : $D$

> Denote the set of $k$ nearest neighbors of $x$ as $S_x$, such that $S_x \in D$ and $|S_x| = k$<br>
> Then ∀ $(x',y') \in D\setminus{}S_x$ and ∀ $(x'',y'') \in S_x$
> $$ dist(x, x') >= max (dist(x, x'')) $$
> That is, every point in $D$ but not in $S_x$, is at least as far away from $x$ as the farthest point in $S_x$ <br>

> We can then define the classifier $h()$ as a function returning the most common label in $S_x$
> $$ h(x) = mode({y'' : (x'',y'') \in S_x}) $$


1.4. <ins>Distance Metric</ins>

The k-NN classifier fundamentally relies on a distance metric to identify the nearest neighbors. The better that metric quantifies similarity, the better the classification.The most common choice is the **Minkowski Distance**

$$ dist(x, z) = (\displaystyle\sum_{i=1}^{d}|x_i-z_i|^{p}) ^{1/p} $$

> p = 1 gives us Manhattan distance<br>
> p = 2 gives us Eucledian distance<br>
> ...

kNN is a non-parametric learner i.e. it does not make assumptions about the form of the mapping function. The entire training data is stored in the model, which becomes slower thus with more data


### 2. Pseudocode

$\text{Classify}(\mathbf{X,Y},x)\;\text{//}\;\mathbf{X}: m\;\text{rows of training data},\;\mathbf{Y}:\text{class labels of}\;\mathbf{X},\;x:\text{test point}$

$\mathbf{for}\;i = 1\;\text{to}\;m\;\mathbf{do}$<br>
$\hspace{3ex}\text{Compute distance}\;d(\mathbf{X}_{i},x)$<br>
$\mathbf{end\;for}$

$\text{Compute set}\;I\;\text{containing indices for the}\;k\;\text{smallest distances in}\;d(\mathbf{X}_{i},x)$<br>
$\mathbf{return}\;\text{majority class label for}\;\{\mathbf{Y_i},\;\text{where}\;i\in I\}$

Execution time for a test point O

### 3. Implementation

Data: The Iris data set contains 3 types of Iris flower with 50 instances each.<br>
Predicted attribute : class of Iris plant<br>
Information attributes : sepal length, sepal width, petal length, petal width

In the following section, we build a KNN classifier in python and make predictions for some test points

In [1]:
# Import libraries

import os
import numpy as np

In [2]:
# Load data

X = np.genfromtxt('datasets\iris.csv', delimiter = ',', dtype = None, skip_header = 1, encoding = 'UTF-8', usecols = [0,1,2,3])
Y = np.genfromtxt('datasets\iris.csv', delimiter = ',', dtype = None, skip_header = 1, encoding = 'UTF-8', usecols = 4)

In [3]:
X.shape

(150, 4)

In [4]:
Y.shape

(150,)

In [5]:
# Split data into train-test

np.random.seed(0)
ids = np.arange(Y.shape[0])
test_ids = np.random.choice(ids, size = 5, replace = False)
train_ids = ids[~np.isin(ids, test_ids)]

X_train = X[train_ids]
Y_train = Y[train_ids]
x_test = X[test_ids]
y_test = Y[test_ids]

In [6]:
# Minkowski distance

def minkowski_dist(v1, v2, p):
    return np.power(np.sum(np.power(abs(v1-v2), p)), 1/p)

# Mode 1-d

def mode1d(v):
    vals, cnt = np.unique(v, return_counts = True)
    return vals[cnt.argmax()]


In [18]:
# kNN

def kNN_classify(X, Y, x, p, k):
    
    dist_arr = np.fromiter((minkowski_dist(v, x, p) for v in X), 'float')
    
    idx_k_smallest = np.argpartition(dist_arr, k)[:k]
    
    return mode1d(Y[idx_k_smallest])


In [8]:
# prediction

pred = np.fromiter((kNN_classify(X_train, Y_train, x, 2, 3) for x in x_test), '<U12')

flag = pred == y_test

accuracy = sum(flag)/len(flag)

accuracy

1.0

### 4. Properties

4.1. <ins> Convergence of 1-NN</ins>

Absract: As $n \rightarrow \infty$, the 1NN error is no more than twice the error of the Bayes Optimal Classifier.

$x_t$ : test point <br>
$x_{NN}$ : nearest neighbor of $x_t$

As $n \rightarrow \infty, \hspace{2ex} dist(x_t, x_{NN}) \rightarrow 0 \; i.e. x_{NN} = x_t$<br>

The bayes optimal classifier provides a theoretical lower bound of error for a given feature respresntation.

Probable error rate of the bayes optimal classifier $\epsilon_{BayesOptCl} \; = \; 1 - P(y=1|x_t)$ 

Probable error rate of 1-NN classifier: $\epsilon_{1NN} \; = \; P(y=1|x_{NN}) \times P(y=0|x_t) + P(y=0|x_{NN}) \times P(y=1|x_t)$

or, $\epsilon_{1NN} \; = \; P(y=1|x_{NN}) \times(1 - P(y=1|x_t)) + (1 - P(y=1|x_{NN})) \times P(y=1|x_t)$

or, $\epsilon_{1NN} \; <= \; (1 - P(y=1|x_t)) + (1 - P(y=1|x_{NN}))$

As $n \rightarrow \infty$, $ x_{NN} = x_t\;$ or, $\epsilon_{1NN} \; <= \; 2(1 - P(y=1|x_t))$

$\therefore \epsilon_{1NN} <= 2\times\epsilon_{BayesOptCl}$


4.2. <ins>Curse of Dimensionality</ins>

Abstract: In high dimensional spaces, the kNN assumption breaks down beacuse points drawn from a probability distribution never tend to be close together.

Let us consider a unit hypercube $[0,1]^d$ which encloses $n$ uniformly sampled training points as 'O'

Let $l$ be the edge length of the smallest hyper-cube 'I' that contains all k-nearest neighbors of a test point.

$\therefore \frac{\text{Volume of inner cube I}}{\text{Volume of outer cube O}} \simeq \frac{\text{training samples within I}}{\text{training samples within O}}$

or, $\dfrac{l^d}{1^d} \simeq \dfrac{k}{n}$

or, $l \simeq (k/n)^{1/d}$

if $n$ = 1000 and $k$ = 10, then $l$ varies with $d$ as:

| d | l |
|:-:|:-:|
| 2 | 0.1|
|10 | 0.631|
|100 | 0.955|
|1000| 0.995|

As $d>>0$, almost the entire space within the unit hypercube is needed to fit the inner hypercube of 10-NN. This implies that the $k$-NN are not particularly closer (and therefore more similar) than any other data points in the training set in a high dimensional space, and thus, the similarity assumption of kNN breaks down.

Although K-NN is hence best suited for low dimensional data, it may still be effective high dimensional spaces if the data lie on a low dimensional subspace or manifold within that higher dimensional space e.g. natural images (digits, faces).

4.3. <ins> kNN for Regression </ins>

The kNN alogorithm can be used for regression (where $y_i \in \mathbb{R}$). The mean of actual values of $k$ nearest neighbors of test point $x_t$ is returned as the predicted value for $x_t$

4.4. <ins> Metric Learning </ins>

The distance metric quantifies similarity between points and helps identify the 'neighbors'. While the Minkowski distance is the most widely used distance metric, kNN becomes truly competitive (higher accuracy) through Metric Learning, where the Mahalanobis distance metric is *learned* from the labeled samples and used to compute distance for kNN classification.

### 5. Closing Notes

k-NN is effective if distance reliably reflect a semantically meaningful notion of similarity. 

As $n \rightarrow \infty$ it is provably very accurate yet very slow. 

As $d>>0$, points drawn from a probability distribution stop being similar to each other and the kNN assumption breaks down